In [1]:
import pandas as pds
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from operator import itemgetter

dataset = pds.read_csv('dataset_yasin.csv')
dataset

,Data Set Yasin
0,Yaa siin
1,"Demi Al Quraan yang penuh hikmah,"
2,"Sesungguhnya kamu salah seorang dari rasul-rasul,"
3,"(yang berada) diatas jalan yang lurus,"
4,(sebagai wahyu) yang diturunkan oleh Yang Maha...
...,...
78,"Katakanlah: ""Ia akan dihidupkan oleh Tuhan yan..."
79,yaitu Tuhan yang menjadikan untukmu api dari k...
80,Dan tidaklah Tuhan yang menciptakan langit dan...
81,Sesungguhnya keadaan-Nya apabila Dia menghenda...


In [2]:
#to lower case
data = dataset["Data Set Yasin"].astype(str).str.lower().str.replace(",","").str.replace(".","").str.replace("(","").str.replace(")","").str.replace(":","").str.replace("\"","")

In [3]:
!pip install Sastrawi #untuk mengubah menjadi bentuk dasar && memisahkan teks
import nltk
nltk.download('punkt')
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import numpy as np
from math import log
carikata = 'Maha suci'

listkata = []
i = 1
for d in data:
    srfactory = StopWordRemoverFactory()
    stopword = srfactory.create_stop_word_remover()
    token = nltk.tokenize.word_tokenize(stopword.remove(str(d)))
    srfactory = StemmerFactory()
    stemmer = srfactory.create_stemmer()
    katadasar = stemmer.stem(str(token))
    listkata.append(katadasar)

     |████████████████████████████████| 215kB 2.7MB/s 
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
N = data.shape[0]
hasilidf = pds.DataFrame(columns=["kata","idf"])
for item in carikata.split():
    dft = np.sum(data.str.contains(item)) # mengecek substring
    idft = log((dft+1),10)
    hasilidf = hasilidf.append(pds.Series([item, idft], index=['kata','idf']),ignore_index=True)

wordList = data.str.split(' ')
maxFt = [len(s)  for s in wordList] #menghitung jumlah data
hasiltf = pds.DataFrame()
for item in carikata.split():
    ft = np.add([s.count(item) for s in wordList], 0)
    ftd = 1 + np.log10(ft)# mengembalikan nilai basis 10 pada log
    hasiltf = hasiltf.append(pds.Series(ftd), ignore_index=True)
    hasiltf = hasiltf.replace(-np.inf, 0)

nilaiakhir = pds.DataFrame()
for i in hasiltf:
    tfidf = hasiltf[i] * hasilidf['idf']
    nilaiakhir = nilaiakhir.append(pds.Series(tfidf), ignore_index=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in log10
  del sys.path[0]


In [5]:
hasilidf

,kata,idf
0,Maha,0.000000
1,suci,0.477121


In [6]:
nilaiakhir

,0,1
0,0.0,0.000000
1,0.0,0.000000
2,0.0,0.000000
3,0.0,0.000000
4,0.0,0.000000
...,...,...
78,0.0,0.000000
79,0.0,0.000000
80,0.0,0.000000
81,0.0,0.000000


In [7]:
N

83

In [8]:
temp = np.transpose(nilaiakhir)
hasilJumlah = pds.DataFrame()
for i in range(N):
  jum = 0
  
  for j in range(dft):
    jum=jum+temp[i][j]

  hasilJumlah = hasilJumlah.append(pds.Series(jum), ignore_index=True)

In [9]:
hasilJumlah.index = nilaiakhir.index+1
hasilJumlah

,0
1,0.000000
2,0.000000
3,0.000000
4,0.000000
5,0.000000
...,...
79,0.000000
80,0.000000
81,0.000000
82,0.000000


In [10]:
hasilJumlah.sort_values(by=[0], inplace=True, ascending=False)
hasilJumlah

,0
83,0.477121
36,0.477121
53,0.000000
61,0.000000
60,0.000000
...,...
27,0.000000
26,0.000000
25,0.000000
24,0.000000


In [11]:
print(hasilJumlah.index.values[0])

83


In [12]:
for i in range (0,20):
  print (str(i+1)+str(".")+dataset.iloc[int (hasilJumlah.index.values[i])-1,0]+str("        ")+(str(hasilJumlah.iloc[i,0])))

1.Maka Maha Suci (Allah) yang di tangan-Nya kekuasaaan atas segala sesuatu dan kepada-Nyalah kamu dikembalikan.        0.47712125471966244
2.Maha Suci Tuhan yang telah menciptakan pasangan-pasangan semuanya, baik dari apa yang ditumbuhkan oleh bumi dan dari diri mereka maupun dari apa yang tidak mereka ketahui.        0.47712125471966244
3.Tidak adalah teriakan itu selain sekali teriakan saja, maka tiba-tiba mereka semua dikumpulkan kepada Kami.        0.0
4.dan hendaklah kamu menyembah-Ku. Inilah jalan yang lurus.        0.0
5.Bukankah Aku telah memerintahkan kepadamu hai Bani Adam supaya kamu tidak menyembah syaitan? Sesungguhnya syaitan itu adalah musuh yang nyata bagi kamu",        0.0
6.Dan (dikatakan kepada orang-orang kafir): "Berpisahlah kamu (dari orang-orang mu'min) pada hari ini, hai orang-orang yang berbuat jahat.        0.0
7.(Kepada mereka dikatakan): "Salam", sebagai ucapan selamat dari Tuhan Yang Maha Penyayang.        0.0
8.Di syurga itu mereka memperoleh buah-buahan d